# Demonstration of GA4GH Variant + Read API

GA4GH examples trimmed from a wider [GA4GH demo](https://github.com/david4096/bioapi-examples/blob/master/python_notebooks/1kg.ipynb)

In [1]:
import ga4gh.client
#baseURL = "http://1kgenomes.ga4gh.org"
baseURL = "http://ga4gh.dursi.ca:8000"
client = ga4gh.client.HttpClient(baseURL)

Data - whether variants, reads, or references - are bundled together in datasets

In [2]:
datasets = list(client.search_datasets())
print("Datasets")
for dataset in datasets:
    print("Name: {}".format(dataset.name))

Datasets
Name: GA4GH_DEM


Here we just have the one dataset, that of the 1kG project; we can set what variantsets are contained within

In [3]:
dataset = client.get_dataset(datasets[0].id)
variantSets = list(client.search_variant_sets(dataset.id))
print("VariantSets")
for variantSet in variantSets:
    print("Name: {}".format(variantSet.name))

VariantSets
Name: phase3-release


In [4]:
variantSetId = variantSets[0].id
variantSet = client.get_variant_set(variantSetId)
print(variantSet.id)

WyJHQTRHSF9ERU0iLCJ2cyIsInBoYXNlMy1yZWxlYXNlIl0


Using that variantid, we can find the callsets associated with the variant set; the callsets belong to a particular sample, called a particular way.  If multiple callers were run, each sample might have two callsets...

In [5]:
callSets = list(client.search_call_sets(variantSetId))
print("CallSets")
for i, callSet in enumerate(callSets):
    print("{}, Name: {}".format(i, callSet.name))
    if i > 4:
        print("... and so on, for a total of {} callsets".format(len(callSets)))
        break

CallSets
0, Name: HG00096
1, Name: HG00533
2, Name: HG00534


We can search within a variantset by genomic region to find variants and calls:

In [6]:
variants = list(client.search_variants(variantSetId, start=10000, end=10300, 
                                       reference_name = "1",
                                       call_set_ids=[callset.id for callset in callSets]))
print("Variants")
for variant in variants:
    print("Reference Name: {}, Start: {}, Name: {}".format(variant.reference_name, 
                                                           variant.start, variant.names[0]))

Variants
Reference Name: 1, Start: 10176, Name: rs367896724
Reference Name: 1, Start: 10234, Name: rs540431307


In [7]:
ex = variants[0]
print(ex.names[0], ex.reference_bases, ex.alternate_bases, ex.info["AF"].values[0])

rs367896724 A ['AC'] string_value: "0.425318986177"



This variant is then called (or not) across all the callsets:

In [8]:
print("Number of calls: {}".format(len(ex.calls)))
print(ex.calls[0])

Number of calls: 3
call_set_name: "HG00096"
call_set_id: "WyJHQTRHSF9ERU0iLCJ2cyIsInBoYXNlMy1yZWxlYXNlIiwiSEcwMDA5NiJd"
genotype: 1
genotype: 0
phaseset: "True"



In [9]:
altcounts = [0,0,0]
for call in ex.calls:
    nalt = call.genotype[0] + call.genotype[1]
    altcounts[nalt] += 1
tot = sum(altcounts)
print("Homozygous ref = {}/{} ({}%)\nHeterozygous alt {}/{} ({}%)\nHomozygous alt = {}/{} ({}%))".\
     format(altcounts[0], tot, int(1000*altcounts[0]*1./tot)/10.,
            altcounts[1], tot, int(1000*altcounts[1]*1./tot)/10.,
            altcounts[2], tot, int(1000*altcounts[2]*1./tot)/10.))

Homozygous ref = 2/3 (66.6%)
Heterozygous alt 1/3 (33.3%)
Homozygous alt = 0/3 (0.0%))


We can find the first call set name corresponding to a hom alt call:

In [10]:
het_alts = [call.call_set_name for call in ex.calls 
            if sum(call.genotype) == 1]
print(het_alts[0])

HG00096


In this case (but not in general!) the readgroupset names are the same as the call set names,
so we can look at the reads corresponding to these calls.  Because reads are aligned to the reference, we need the reference id.  Note too that we have to go through all the read group sets to find the one we want

In [11]:
reference_set = list(client.search_reference_sets())[0]
references = [r for r in client.search_references(reference_set_id=reference_set.id)]
print(references[0])

read_group_sets = list(client.search_read_group_sets(dataset_id=dataset.id))
read_group_set = [rgs for rgs in read_group_sets if rgs.name in het_alts][0]
print("Read group set : {}".format(read_group_set.name))

id: "WyJHUkNoMzciLCIxIl0"
length: 138395
md5checksum: "bb07c91cda4645ad8e75e375e3d6e5eb"
name: "1"

Read group set : HG00096


In [12]:
start = ex.start-10
end = ex.end+10
length = end-start+1

print(client.list_reference_bases(references[0].id, start=start, end=end))
print("")
for read_group in read_group_set.read_groups:
    sequences = list(client.search_reads(read_group_ids=[read_group.id], 
                                        start=start, end=end,
                                        reference_id=references[0].id))
    for sequence in sequences:
        seqstart = sequence.alignment.position.position
        pos_strand = sequence.alignment.position.strand == 2
        if seqstart <= start and pos_strand:
            print(sequence.aligned_sequence[start-seqstart:start-seqstart+length])

CCTAACCCTAACCTAACCCTA

CCTAACCCTAACCCTAACCCTA
ACCCTAACCCTAACCCTAAACC
CCTAACCCTAACCTAACCCTAA
CCTAACCCAACCCTAACCCTAA
CCTAACCCTAACCCTGACCCTA
CCTAACCCTAACCTAACCCTAA
CCTAACCCTAACCCCAACCCCA


## ExAC

ExAC also has a [rest API](https://github.com/hms-dbmi/exac_browser), but it only handles variants and their annotations, and there isn't a cute python API wrapper yet:

In [13]:
import requests
exac_base_url = "http://exac.hms.harvard.edu/rest/"
chrom = "1"
start = "13000"
stop = "20000"
region_id = chrom + "-" + start + "-" + stop
print(region_id)

1-13000-20000


In [14]:
response = requests.get(
        exac_base_url + "region/variants_in_region/" + region_id)
variants = response.json()
print(variants[0])

print("Variant allele frequencies: ")
print(sorted([variant['allele_count'] for variant in variants]))

{'allele_num': 8432, 'pop_acs': {'Other': 0, 'European (Non-Finnish)': 0, 'South Asian': 2, 'African': 0, 'European (Finnish)': 0, 'Latino': 0, 'East Asian': 0}, 'allele_count': 2, 'quality_metrics': {'QD': '23.42', 'BaseQRankSum': '0.727', 'MQ': '35.72', 'DP': '139843', 'VQSLOD': '-1.687e+00', 'ReadPosRankSum': '0.727', 'MQRankSum': '0.727', 'InbreedingCoeff': '-0.0844', 'ClippingRankSum': '1.15', 'FS': '0.000'}, 'rsid': '.', 'pop_homs': {'Other': 0, 'European (Non-Finnish)': 0, 'South Asian': 1, 'African': 0, 'European (Finnish)': 0, 'Latino': 0, 'East Asian': 0}, 'alt': 'C', 'pop_ans': {'Other': 90, 'European (Non-Finnish)': 2116, 'South Asian': 5052, 'African': 770, 'European (Finnish)': 16, 'Latino': 134, 'East Asian': 254}, 'flags': [], 'allele_freq': 0.00023719165085388995, 'major_consequence': 'splice_region_variant', 'HGVSp': '', 'chrom': '1', 'filter': 'PASS', 'hom_count': 1, 'HGVSc': 'n.412G>C', 'CANONICAL': '', 'ref': 'G', 'HGVS': 'n.412G>C', 'variant_id': '1-13372-G-C', 'p

It also has somewhat wider search functionality, with syntactic sugar for searching by gene/transcript, or prioritizing (but not filtering) responses by consenquence:

In [15]:
response = requests.get(
            exac_base_url + "variant/consequences/"+"20-76735-A-T")
variants = response.json()
print(variants['missense_variant']['ENSG00000178591'][0]['PolyPhen'])

possibly_damaging(0.693)


In [16]:
response = requests.get(
            exac_base_url + "awesome?query="+"PCSK9"+"&service=variants_in_gene")
variants = response.json()
print(variants[0])

{'allele_num': 30642, 'pop_acs': {'Other': 0, 'European (Non-Finnish)': 1, 'South Asian': 0, 'African': 0, 'European (Finnish)': 0, 'Latino': 0, 'East Asian': 0}, 'allele_count': 1, 'quality_metrics': {'QD': '5.42', 'BaseQRankSum': '-1.093e+00', 'MQ': '60.00', 'DP': '695607', 'VQSLOD': '-2.094e+00', 'ReadPosRankSum': '-3.120e-01', 'MQRankSum': '-8.110e-01', 'InbreedingCoeff': '-0.0361', 'ClippingRankSum': '0.555', 'FS': '2.276'}, 'rsid': '.', 'pop_homs': {'Other': 0, 'European (Non-Finnish)': 0, 'South Asian': 0, 'African': 0, 'European (Finnish)': 0, 'Latino': 0, 'East Asian': 0}, 'alt': 'T', 'pop_ans': {'Other': 204, 'European (Non-Finnish)': 14444, 'South Asian': 8684, 'African': 3508, 'European (Finnish)': 360, 'Latino': 1110, 'East Asian': 2332}, 'flags': [], 'allele_freq': 3.2634945499641017e-05, 'major_consequence': '5_prime_UTR_variant', 'HGVSp': '', 'chrom': '1', 'filter': 'PASS', 'hom_count': 0, 'HGVSc': 'c.-36C>T', 'CANONICAL': 'YES', 'ref': 'C', 'HGVS': '', 'variant_id': '1